In [1]:
from google.colab import files
import pandas as pd

# Upload
uploaded = files.upload()

# Get filename
filename = list(uploaded.keys())[0]

# Load CSV
df_spam = pd.read_csv(filename, encoding='latin-1')  # if utf-8 fails

# Preview
df_spam.head()

Saving spam.csv to spam.csv


,text,target
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [2]:
!pip install transformers

In [3]:
from transformers import pipeline

# Load a text-generation model (e.g. GPT2)
classifier = pipeline("text-generation", model="gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


text-generation LLM with prompt **engineering**

In [5]:
def classify_email(email_text):
    prompt = f"Classify this email as 'spam' or 'ham':\n\n{email_text}\n\nAnswer with only 'spam' or 'ham'."
    result = classifier(prompt, max_length=50, do_sample=False)
    return result[0]['generated_text'].split(":")[-1].strip().split("\n")[0]

In [7]:
print(df_spam.columns)


Index(['text', 'target'], dtype='object')


In [10]:
from transformers import pipeline

# Load zero-shot-classification pipeline (optimized for classification)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define classification function with candidate labels
def classify_email_zeroshot(email_text):
    candidate_labels = ["spam", "ham"]
    result = classifier(email_text, candidate_labels, truncation=True)
    return result['labels'][0]

# Apply to a small sample first to test and avoid long runtime
sample_df = df_spam.sample(n=20, random_state=42).copy()
sample_df['predicted_label'] = sample_df['target'].apply(classify_email_zeroshot)

# Preview results
sample_df[['target', 'predicted_label']]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


,target,predicted_label
3245,ham,ham
944,ham,ham
1044,spam,spam
2484,ham,ham
812,spam,spam
2973,ham,ham
2991,ham,ham
2942,ham,ham
230,ham,ham
1181,ham,ham


In [14]:
df_spam.to_csv("spam_email_classification_results.csv", index=False)